In [18]:
!pip install rich pydantic python-dotenv langchain_openai langchain arxiv json_repair bs4 langchain_community click markdown lxml

  Using cached lxml-5.3.1-cp311-cp311-win_amd64.whl.metadata (3.8 kB)
Using cached lxml-5.3.1-cp311-cp311-win_amd64.whl (3.8 MB)



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from typing import List
from pydantic import BaseModel, Field
from gpt_researcher import GPTResearcher
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
import asyncio
from datetime import datetime
from dotenv import load_dotenv
import nest_asyncio
from rich.markdown import Markdown
from rich.console import Console

load_dotenv()
console = Console()

class ResearchContext(BaseModel):
    """Model to store research context and metadata"""
    query: str = Field(..., description="The query used for research")
    timestamp: datetime = Field(default_factory=datetime.now, description="Timestamp of when research was conducted")
    report: str = Field(..., description="The research report")
    sources: List[dict] = Field(..., description="List of research sources URLs")
    costs: float = Field(..., description="research costs")
    images: List[str] = Field(..., description="List of research images URLs")

class QueryDecision(BaseModel):
    """Model for the decision on whether to use existing context or perform new research"""
    use_existing_context: bool = Field(description="Whether to use existing research context or not")
    reasoning: str = Field(description="Reasoning behind the decision")
    relevant_context_indices: List[int] = Field(
        default=[], description="List of integer indices referencing the context data used for the decision"
    )

class ChatMemory:
    def __init__(self, max_contexts: int = 10):
        self.contexts: List[ResearchContext] = []
        self.max_contexts = max_contexts
    
    def add_context(self, context: ResearchContext):
        """Add new research context and maintain size limit"""
        self.contexts.append(context)
        if len(self.contexts) > self.max_contexts:
            self.contexts.pop(0)
    
    def get_all_contexts(self) -> List[ResearchContext]:
        return self.contexts

class ResearchChatbot:
    def __init__(self):
        self.memory = ChatMemory()
        self.llm = ChatOpenAI(temperature=0, model="chatgpt-4o-latest")
    
    async def _decide_research_need(self, query: str) -> QueryDecision:
        contexts = self.memory.get_all_contexts()
        
        if not contexts:
            return QueryDecision(
                use_existing_context=False,
                reasoning="No existing research context available"
            )
        
        context_summaries = [
            f"Context {i}:\n"
            f"📝 Query: '{ctx.query}'\n"
            f"⏰ Timestamp: {ctx.timestamp}\n"
            f"📑 Summary: {ctx.report[:1000]}..."
            for i, ctx in enumerate(contexts)
        ]
        
        context_summary_str = '\n\n'.join(context_summaries)
        
        parser = PydanticOutputParser(pydantic_object=QueryDecision)
        
        prompt = ChatPromptTemplate.from_messages([
            (
                "system",
                """You are a research assistant helping to determine if existing research can answer a query or if new research is needed.

                Analyze the following research contexts and the new query carefully. Consider:
                1. How recent and relevant is the existing research?
                2. Does it fully address all aspects of the new query?
                3. Has the information likely changed since the research was conducted?

                Available research contexts:
                {context_summaries}

                {format_instructions}
                
                Provide detailed reasoning for your decision.
                """
            ),
            ("human", "New research query: {query}")
        ]).partial(format_instructions=parser.get_format_instructions())
        
        chain = prompt | self.llm | parser
        
        try:
            result = await chain.ainvoke({
                "query": query,
                "context_summaries": context_summary_str
            })
            return result
        except Exception as e:
            return QueryDecision(
                use_existing_context=False,
                reasoning=f"Error parsing LLM response: {str(e)}"
            )
    
    async def _generate_response_from_context(
        self, query: str, relevant_contexts: List[ResearchContext]
    ) -> str:
        context_text = "\n\n".join([
            f"### Research for '{ctx.query}':\n"
            f"{ctx.report}\n"
            # f"**Sources**: {', '.join(ctx.sources)}"
            for ctx in relevant_contexts
        ])
        
        prompt = ChatPromptTemplate([
            (
                "system",
                """You are a knowledgeable research assistant. Using the provided research context, 
                create a comprehensive response in markdown format. Include:
                
                1. A clear and direct answer to the query
                2. Relevant supporting details and examples
                3. Citations when referencing specific information
                4. A brief summary if the response is lengthy
                
                Research Context:
                {context}
                """
            ),
            ("human", "{query}")
        ])

        chain = prompt | self.llm
        
        response = await chain.ainvoke({
            'context': context_text,
            'query': query
        })
        
        return response

    async def chat(self, query: str) -> str:
        decision = await self._decide_research_need(query)
        
        console.print("\n💭 [italic blue]Thinking...[/italic blue]")
        
        if decision.use_existing_context:
            console.print(f"\n📚 [green]Using existing research: {decision.reasoning}[/green]")
            relevant_contexts = [
                self.memory.contexts[i]
                for i in decision.relevant_context_indices
            ]
            response = await self._generate_response_from_context(query, relevant_contexts)
            response = response.content
        else:
            console.print(f"\n🔍 [yellow]Conducting new research: {decision.reasoning}[/yellow]")
            researcher = GPTResearcher(query, "research_report", config_path='config.json')
            research_result = await researcher.conduct_research()
            response = await researcher.write_report()
            
            context = ResearchContext(
                query=query,
                timestamp=datetime.now(),
                report=response,
                sources=researcher.get_research_sources(),
                costs=researcher.get_costs(),
                images=researcher.get_research_images()
            )
            self.memory.add_context(context)
        
        # Format response in markdown
        console.print("\n[bold green]Response:[/bold green]")
        console.print(Markdown(response))
        
        if not decision.use_existing_context:
            console.print("\n📊 [bold]Research Statistics:[/bold]")
            console.print(f"💰 Cost: ${context.costs:.3f}")
            console.print(f"📚 Sources: {len(context.sources)}")
            if context.images:
                console.print(f"🖼️ Images: {len(context.images)}")
        
        return response

async def main():
    chatbot = ResearchChatbot()
    console.print("[bold blue]Research Chatbot[/bold blue]")
    console.print("Type 'exit' to end the conversation\n")
    
    while True:
        query = input("❓ [bold]Your question:[/bold] ")
        if query.lower() == 'exit':
            break
            
        try:
            await chatbot.chat(query)
        except Exception as e:
            console.print(f"\n[red]Error: {str(e)}[/red]")
        
        console.print("\n---\n")

if __name__ == "__main__":
    nest_asyncio.apply()
    asyncio.run(main())

Research Chatbot

Type 'exit' to end the conversation

💭 Thinking...

🔍 Conducting new research: No existing research context available

INFO:     [21:33:39] 🔍 Starting the research task for 'Cars 2025'...
INFO:     [21:33:39] 🚗 Automotive Trends Agent
INFO:     [21:33:39] 🌐 Browsing the web to learn more about the task: Cars 2025...
INFO:     [21:33:41] 🤔 Planning the research strategy and subtasks...
INFO:     [21:33:51] 🗂️ I will conduct my research based on the following queries: ['Car and Driver 10Best Cars for 2025: full list and detailed reviews', '2025 Hyundai Santa Fe vs. Toyota Camry Hybrid: comparing features and performance', 'Upcoming electric and hybrid cars in 2025: BMW i3, VW ID.2, and Toyota Celica rumors', 'J.D. Power and Edmunds rankings of best new car deals and incentives for 2025 models', 'Cars 2025']...
INFO:     [21:33:51] 
🔍 Running research for 'Car and Driver 10Best Cars for 2025: full list and detailed reviews'...
INFO:     [21:33:51] 
🔍 Running research for '2025 Hyundai Santa Fe vs. Toyota Camry Hybrid: comparing features and performance'...
INFO:     [21:33:51] 
🔍 Running research for 'Upc

...

📊 Research Statistics:

💰 Cost: $0.126

📚 Sources: 15

🖼️ Images: 7

---